In [32]:
from datetime import datetime
import xmltodict
import pickle
import xml.etree.ElementTree as xml
import os

## 1 Obtendo dados dos arquivos xml

Temos muitas informações relevantes para analizar o negócio armazenadas localmente em forma de `.xml`s que podemos transformar em dados tabulares.

### 1.1 Notas de vendas

A primeira fonte de dados que vamos explorar são as notas de vendas. Vamos começar com um gerador que nos entregará o nome dos arquivos relevantes:


In [33]:
PASTA_VENDAS = r"C:\GDOOR Sistemas\GDOOR SLIM\NFCe\XML\Vendas"
nfes = [os.path.join(PASTA_VENDAS, filename) for filename in os.listdir(PASTA_VENDAS) if len(filename) > 10]

In [34]:
def get_pay(xml: dict):
    """return the payment section of the `xml`"""
    try:
        pag = xml["NFe"]["infNFe"]["total"]["ICMSTot"]["pag"]
    except KeyError:
        try:
            pag = xml["nfeProc"]["NFe"]["infNFe"]["pag"]
        except KeyError:
            pag = xml["NFe"]["infNFe"]["pag"]
    if type(pag["detPag"]) is list:
        return {
            "type": ";".join([x["tPag"] for x in pag["detPag"]]),
            "amount": ";".join([x["vPag"] for x in pag["detPag"]])}
    else:
        return {
            "type": pag["detPag"]["tPag"],
            "amount": pag["detPag"]["vPag"]}

incs = []
_ls = []
for nf in nfes:
    with open(nf) as doc:
        xmlelem = xmltodict.parse(doc.read())
    try:
        _ls.append(get_pay(xmlelem))
    except KeyError:
        incs.append(xmlelem)

len(incs)

0

In [35]:
[a for a in _ls if len(a["type"]) > 2]

[{'type': '01;04', 'amount': '35.50;54.28'},
 {'type': '03;99', 'amount': '270.00;1.38'},
 {'type': '01;04', 'amount': '190.00;24.50'},
 {'type': '03;01', 'amount': '60.00;2.00'},
 {'type': '01;99', 'amount': '50.00;72.47'},
 {'type': '01;99', 'amount': '0.20;45.00'},
 {'type': '01;01', 'amount': '4.00;7898286201937.00'},
 {'type': '01;99', 'amount': '30.00;120.40'},
 {'type': '99;01', 'amount': '154.50;0.03'},
 {'type': '99;01', 'amount': '50.00;0.20'},
 {'type': '01;99', 'amount': '10.00;24.50'},
 {'type': '01;99', 'amount': '4.00;44.70'},
 {'type': '01;99', 'amount': '0.03;146.20'},
 {'type': '01;99', 'amount': '0.04;124.20'},
 {'type': '01;99', 'amount': '40.00;11.80'},
 {'type': '01;99', 'amount': '0.03;78.30'},
 {'type': '01;99', 'amount': '0.02;22.90'},
 {'type': '01;01', 'amount': '2.00;7896098900109.00'},
 {'type': '01;99', 'amount': '60.00;137.60'},
 {'type': '99;01', 'amount': '20.20;20.00'},
 {'type': '01;99', 'amount': '30.00;139.71'},
 {'type': '99;01', 'amount': '6.00;20

Se a célula anterior retornou `0`, conseguiremos processar as informações de pagamento de todos os documentos.

Agora só faltam a **data de emissão** e os **números de identificação** das notas de venda: 

In [36]:
def get_key(xml: dict):
    try:
        return xml["nfeProc"]["NFe"]["infNFe"]["@Id"][3:]
    except KeyError:
        return xml["NFe"]["infNFe"]["@Id"][3:]

incs = []
for nf in nfes:
    with open(nf) as doc:
        xmlelem = xmltodict.parse(doc.read())
    try:
        _ = get_key(xmlelem)
    except KeyError:
        incs.append(xmlelem)

len(incs)

0

Se a célula acima retornou `0`, significa que agora só precisamos fazer o *parsing* das **datas de emissão**:

In [37]:
def get_dt(xml: dict):
    try:
        return xml["nfeProc"]["NFe"]["infNFe"]["ide"]["dhEmi"]
    except KeyError:
        return xml["NFe"]["infNFe"]["ide"]["dhEmi"]

incs = []
for nf in nfes:
    with open(nf) as doc:
        xmlelem = xmltodict.parse(doc.read())
    try:
        _ = get_dt(xmlelem)
    except KeyError:
        incs.append(xmlelem)

len(incs)

0

In [38]:
with open(nfes[5000]) as doc:
    xmlelem = xmltodict.parse(doc.read())

In [39]:
def get_total(xml: dict):
    try:
        total = xml["nfeProc"]["NFe"]["infNFe"]["total"]["ICMSTot"]
    except KeyError:
        total = xml["NFe"]["infNFe"]["total"]["ICMSTot"]
    
    products = None
    if "vNF" in total.keys():
        products = total["vNF"]
    
    discount = None
    if "vDesc" in total.keys():
        discount = total["vDesc"]

    taxes = None
    if "vTotTrib" in total.keys():
        taxes = total["vTotTrib"]

    return {
        "products": products,
        "discount": discount,
        "taxes": taxes}


incs = []
for nf in nfes:
    with open(nf) as doc:
        xmlelem = xmltodict.parse(doc.read())
    try:
        _ = get_total(xmlelem)
    except KeyError:
        incs.append(xmlelem)

len(incs)

0

In [40]:
def parse_(xml: dict):
    key = get_key(xml)
    dt = get_dt(xml)
    pay = get_pay(xml)
    total = get_total(xml)

    return {"key":key, "dt":dt, "pay":pay, "total": total}

parsed = []
incs = []
for nf in nfes:
    with open(nf) as doc:
        xmlelem = xmltodict.parse(doc.read())
    try:
        vals = parse_(xml=xmlelem)
        parsed.append(vals)
    except KeyError:
        incs.append(xmlelem)

In [44]:
[a for a in parsed if a["total"]["taxes"] is not None]

[]

In [42]:
with open(nfes[-1]) as doc:
    xmlelem = xmltodict.parse(doc.read())

In [43]:
parsed = []
for nf in nfes:
    dte = datetime.strptime(xmlelem["nfeProc"]["NFe"]["infNFe"]["ide"]["dhEmi"], "%Y-%m-%dT%H:%M:%S%z")
    print(dte, end="\r")

KeyError: 'nfeProc'